Import Necessary Libraries

In [105]:
import statsapi as api
import pybaseball as pyb
import pandas as pd
import tempfile
import os
import re
import requests
from bs4 import BeautifulSoup, SoupStrainer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import date as d
from datetime import timedelta

Input Player Name and Date


NEED TO CHECK AND FIX ALL CELLS: IF STATS FOR SOMETHING DO NOT EXIST NEED TO DEAL WITH THAT!

ALSO NEED TO ENSURE CORRECT VALUES ARE PASSED IN FOR COMPUTING CORRECT VALUES

NEED TO CHECK PLAYER AVG OVER X GAMES - NEED TO COMPUTE PLAYER AVG OVER LAST GAME FOR PAST SEASONS AS WELL

NEED TO ENSURE THAT ALL DATA IS OF CORRECT TYPE, CORRECT VALUES, CORRECT YEARS, AND IF NONE NEEDS TO BE HANDLED

FIX CURRENT SET UP FOR COMPUTING AVERAGES. SHIT IS BUGGY AS FUCK

IF IT APPEARS TO BE GOOD: THERE WILL BE :P NEXT TO IT. IF NEEDS ADJUSTMENTS, VERIFICATION, OR REWORK, :( NEXT TO IT

ESPECIALLY DECIDE HOW TO DEAL WITH EDGE CASES AND DIVISION BY 0.

WALKS, HIT BY PITCH, AND SAC FLIES DO NOT COUNT AS AN AT BAT

TOTAL BASES ARE ONLY BASES REACHED VIA HITS.

In [106]:
first_name = input("Enter Player First Name: ")
last_name = input('Enter Player Last Name: ')
#date = input('Enter Date: ')
date = '2024-06-21'
year = int(date[:4])
last_year = year - 1
print(year)
player_name = f'{first_name.lower()} {last_name.lower()}'


2024


fetch player_id :P

In [107]:
player_info = api.lookup_player(player_name, season=year)
player_id = player_info[0]['id']
team_id = player_info[0]['currentTeam']['id']
print(player_id)
print(team_id)

621043
142


Gather batter_hand, batter_age, , batter_strike_zone_top, batter_strike_zone_bottom :P

In [108]:
person_info = api.get('people',{'personIds':player_id, 'season':year,'sportId':1})
batter_hand = person_info['people'][0]['batSide']['code']
batter_age = person_info['people'][0]['currentAge']
batter_strike_zone_top = person_info['people'][0]['strikeZoneTop']
batter_strike_zone_bottom = person_info['people'][0]['strikeZoneBottom']
batter_starting_year = int(person_info['people'][0]['mlbDebutDate'][:4])
batter_year = year - batter_starting_year


print(batter_age)
print(batter_year)
print(batter_hand)
print(batter_strike_zone_top)
print(batter_strike_zone_bottom)

29
9
R
3.6
1.72


fetch game_pk, home, opp_id, venue_id, sPitcher_id, sPitcher_hand :P

In [109]:
game_info = api.schedule(date=date,team=team_id,sportId=1)
game_pk = game_info[0]['game_id']
home_team_id = game_info[0]['home_id']
if home_team_id == team_id:
    home = True
    opp_id = game_info[0]['away_id']
    sPitcher_name = game_info[0]['away_probable_pitcher']
else:
    home = False
    opp_id = game_info[0]['home_id']
    sPitcher_name = game_info[0]['home_probable_pitcher']

sPitcher_info = api.lookup_player(sPitcher_name)
sPitcher_id = sPitcher_info[0]['id']
sPitcher_info_2 = api.get('people',{'personIds':sPitcher_id, 'season':year,'sportId':1})
sPitcher_hand = sPitcher_info_2['people'][0]['pitchHand']['code']

venue_id = game_info[0]['venue_id']

if game_info[0]['status'] == 'Final':
    game_status = True
else:
    game_status = False

print(game_status)
print(game_pk)
print(opp_id)
print(sPitcher_name)
print(sPitcher_id)
print(sPitcher_hand)
print(venue_id)
print(home)

False
745649
133
Joey Estes
683155
R
10
False


fetch player game stats: temp, wind_speed, hp_ump_id, game_time. Fetch list of proj_ump_ids if Home Plate Umpire not yet specified :(

What should happen when this data isnt uploaded yet?

In [110]:
game_boxscore_info = api.boxscore_data(gamePk=game_pk)

temp = None
wind_speed = None
game_time = None

for x in game_boxscore_info['gameBoxInfo']:
    if x.get('label') == 'Weather':
        temp_info = x['value']
        temps = re.findall(r'\d+', temp_info)
        if temps:
            temp = int(temps[0])
            

    if x.get('label') == 'Wind':
        wind_speed_info = x['value']
        winds = re.findall(r'\d+', wind_speed_info)
        if winds:
            wind_speed = int(winds[0])
            

    if x.get('label') == 'First pitch':
        game_time = x['value']
        

    if x.get('label') == 'Umpires':
        ump_info = x['value']
        if 'HP:' in ump_info:
            pattern = r'HP: (.*?)\.'
            match = re.search(pattern, ump_info)
            if match:
                hp_ump = match.group(1)
                hp_ump_info = api.get('jobs_umpires', {})
                for person in hp_ump_info['roster']:
                    if person['person'].get('fullName') == hp_ump:
                        hp_ump_id = person['person'].get('id')
                        print(hp_ump_id)

        else:
            umps = re.findall(r'[\w\s]+', ump_info)
            umps = [ump.strip() for ump in umps if ump.strip]
            ump_list = list(umps) + [None]*(4-len(umps))
            umps_tuple = tuple(ump_list[:4])
            proj_ump_ids = []
            for ump in umps_tuple:
                ump_info = api.get('jobs_umpires',{})
                for person in ump_info['roster']:
                    if person['person'].get('fullName') == ump:
                        ump_id = person['person'].get('id')
                        proj_ump_ids.append(ump_id)
            print(proj_ump_ids)

print(temp)
print(wind_speed)
print(game_time)



521889
63
10
6:41 PM.


fetch player game data: :P
- batting position

In [111]:
game_content = api.boxscore_data(gamePk=game_pk)
if home == True:
    batters_info = game_content['homeBatters']
else:
    batters_info = game_content['awayBatters']

batting_position = None

for batter in batters_info:
    if batter.get('personId') == player_id:
        batting_position = int(batter.get('battingOrder')[0])

print (batting_position)

2


fetch game player stats :P
- If game_status == True
    - gather player stats
- If game_status == False
    - set player stats to None

In [112]:
if game_status == True:
    game_stats_info = api.get('person_stats',{'personId':player_id,'gamePk':game_pk})
    game_stats = game_stats_info['stats'][0]['splits'][2]['stat']
    ab = game_stats.get('atBats')
    h = game_stats.get('hits')
    r = game_stats.get('runs')
    rbi = game_stats.get('rbi')
    hr = game_stats.get('homeRuns')
    tb = game_stats.get('totalBases')
    k = game_stats.get('strikeOuts')
    bb = game_stats.get('baseOnBalls')

else:
    ab = None
    h = None
    r = None
    rbi = None
    hr = None
    tb = None
    k = None
    bb = None


print(f'ab: {ab}')
print(f'h: {h}')
print(f'r: {r}')
print(f'rbi: {rbi}')
print(f'hr: {hr}')
print(f'tb: {tb}')
print(f'k: {k}')
print(f'bb: {bb}')


ab: None
h: None
r: None
rbi: None
hr: None
tb: None
k: None
bb: None


Determine Whether Most Recent Streak Averages, or Historical Streak Averages are Needed for Player

Format Player URL for baseballreference.com

In [113]:
initial = last_name[:1].lower()

full_name = f'{first_name} {last_name}'.lower()

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                formatted_player_key = player_link[11:20]

print(formatted_player_key)

correca01


Collect Data for Games Played Up to Desired Date
    - For entirely current year, or entirely last year

Format Player Game Data URL

In [114]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

if date == season_start_date:
    date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'
else:
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={year}#batting_gamelogs'

Set Correct Date

In [115]:
day = int(date[8:10])
month = int(date[5:7])

date_dt = d(year, month, day)

new_date_dt = date_dt - timedelta(days=1)

date = new_date_dt.strftime("%Y-%m-%d")

day = int(date[8:10])
month = int(date[5:7])

print(date)
print(day)
print(month)

2024-06-20
20
6


Scrape Player Game Data

In [116]:
response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    df = pd.DataFrame(data)

    df.rename(columns={
        'batting_avg': 'BA',
        'onbase_perc': 'OBP',
        'slugging_perc': 'SLG',
    }, inplace=True)

    all_columns = df.columns.tolist()

    columns_to_keep = ['game_num', 'date_game', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

    columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

    correct_columns_df = df.drop(columns_to_drop, axis=1)

    columns_to_convert_int = ['game_num', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

    columns_to_convert_float = ['BA', 'OBP', 'SLG']

    for column in columns_to_convert_int:
         correct_columns_df[column] = correct_columns_df[column].astype(int)

    for column in columns_to_convert_float:
        correct_columns_df[column] = correct_columns_df[column].astype(float)

    correct_columns_df['TB'] = correct_columns_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

    column_order = ['game_num', 'date_game', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
    correct_columns_df = correct_columns_df[column_order]

print(correct_columns_df)


    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
0          1  2024-03-28   4  3  0    2   1   0   0   4   1   0  0.750  0.800   
1          2  2024-03-30   5  0  0    0   0   0   0   0   0   0  0.333  0.400   
2          3  2024-03-31   2  0  0    0   0   0   0   0   0   1  0.273  0.333   
3          4  2024-04-02   1  0  1    0   0   0   0   0   3   1  0.250  0.438   
4          5  2024-04-03   3  1  2    1   0   0   0   1   2   1  0.267  0.476   
5          6  2024-04-04   4  3  0    0   1   0   0   4   0   1  0.368  0.520   
6          7  2024-04-06   3  1  1    0   0   0   0   1   2   1  0.364  0.533   
7          8  2024-04-08   4  0  0    0   0   0   0   0   0   0  0.308  0.471   
8          9  2024-04-09   4  1  1    1   0   0   1   4   0   2  0.300  0.447   
9         10  2024-04-10   4  2  0    0   0   0   0   2   0   1  0.324  0.452   
10        11  2024-04-12   2  0  0    0   0   0   0   0   0   1  0.306  0.432   
11        12  2024-04-29   4

Season Averages Up to Desired Date

In [117]:
season_stats = {}

game_data = correct_columns_df.iloc[-1]

games_played = game_data['game_num']

print(games_played)

season_stats['ab_per_game'] = correct_columns_df['AB'].mean()
season_stats['h_per_game'] = correct_columns_df['H'].mean()
season_stats['r_per_game'] = correct_columns_df['R'].mean()
season_stats['rbi_per_game'] = correct_columns_df['RBI'].mean()
season_stats['hr_per_game'] = correct_columns_df['HR'].mean()
season_stats['tb_per_game'] = correct_columns_df['TB'].mean()
season_stats['bb_per_game'] = correct_columns_df['BB'].mean()
season_stats['k_per_game'] = correct_columns_df['SO'].mean()
season_stats['ba_szn'] = game_data['BA']
season_stats['obp_szn'] = game_data['OBP']
season_stats['slg_szn'] = game_data['SLG']

season_stats_df = pd.DataFrame([season_stats])

print(season_stats_df)

57
   ab_per_game  h_per_game  r_per_game  rbi_per_game  hr_per_game  \
0     3.701754    1.140351    0.631579      0.614035     0.140351   

   tb_per_game  bb_per_game  k_per_game  ba_szn  obp_szn  slg_szn  
0     1.859649     0.403509    0.754386   0.308     0.37    0.502  


Streak Average Data Collection

In [118]:
if games_played < 30:
    last_season_end_date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']

    day = int(last_season_end_date[8:10])
    month = int(last_season_end_date[5:7])

    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'

    response = requests.get(search_url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

    if table_rows:

        data = []

        for row in table_rows:

            row_data = {}

            th = row.find('th')
            
            row_num = th.text
            
            row_data['game_num'] = row_num

            for td in row.find_all('td'):

                stat_type = td['data-stat']

                if stat_type == 'date_game':
                    stat_value = td['csk'][:10]
                else:
                    stat_value = td.text

                row_data[stat_type] = stat_value

            if row_data:

                game_day = int(row_data['date_game'][8:10])
                game_month = int(row_data['date_game'][5:7])

                if game_month < month:
                        data.append(row_data)
                elif game_month == month:
                    if game_day <= day:
                        data.append(row_data)

        df = pd.DataFrame(data)

        df.rename(columns={
            'batting_avg': 'BA',
            'onbase_perc': 'OBP',
            'slugging_perc': 'SLG',
        }, inplace=True)

        all_columns = df.columns.tolist()

        columns_to_keep = ['game_num', 'date_game', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

        columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

        last_season_df = df.drop(columns_to_drop, axis=1)

        columns_to_convert_int = ['game_num', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

        columns_to_convert_float = ['BA', 'OBP', 'SLG']

        for column in columns_to_convert_int:
            last_season_df[column] = last_season_df[column].astype(int)

        for column in columns_to_convert_float:
            last_season_df[column] = last_season_df[column].astype(float)

        last_season_df['TB'] = last_season_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

        column_order = ['game_num', 'date_game', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
        last_season_df = last_season_df[column_order]

    print(last_season_df)




Last 7 Averages:

In [119]:
if games_played < 7:
    carry_over_games = 7 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_7_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_7_game_num = games_played - 7
    last_7_df = correct_columns_df.iloc[last_7_game_num:]

print(last_7_df)

last_7_stats = {}

last_7_stats['ab_last_7'] = last_7_df['AB'].mean()
last_7_stats['h_last_7'] = last_7_df['H'].mean()
last_7_stats['r_last_7'] = last_7_df['R'].mean()
last_7_stats['rbi_last_7'] = last_7_df['RBI'].mean()
last_7_stats['hr_last_7'] = last_7_df['HR'].mean()
last_7_stats['tb_last_7'] = last_7_df['TB'].mean()
last_7_stats['bb_last_7'] = last_7_df['BB'].mean()
last_7_stats['k_last_7'] = last_7_df['SO'].mean()
last_7_stats['ba_last_7'] = round(last_7_df['H'].sum() / last_7_df['AB'].sum(), 3)
last_7_stats['obp_last_7'] = round((last_7_df['H'].sum() + last_7_df['BB'].sum() + last_7_df['HBP'].sum()) / last_7_df['AB'].sum(), 3)
last_7_stats['slg_last_7'] = round( last_7_df['TB'].sum() / last_7_df['AB'].sum(), 3)

last_7_avg_df = pd.DataFrame([last_7_stats])

print(last_7_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
50        51  2024-06-13   4  3  2    3   0   0   1   6   0   0  0.299  0.363   
51        52  2024-06-14   4  1  2    0   0   0   0   1   1   0  0.298  0.364   
52        53  2024-06-16   5  3  2    3   0   0   2   9   0   1  0.306  0.369   
53        54  2024-06-16   5  2  0    1   0   0   0   2   0   1  0.308  0.370   
54        55  2024-06-18   4  3  2    0   1   0   0   4   1   0  0.317  0.379   
55        56  2024-06-19   5  0  0    0   0   0   0   0   0   0  0.310  0.371   
56        57  2024-06-20   1  0  0    0   0   0   0   0   0   0  0.308  0.370   

      SLG  HBP  
50  0.481    0  
51  0.476    0  
52  0.510    0  
53  0.507    0  
54  0.517    0  
55  0.505    0  
56  0.502    0  
   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0        4.0  1.714286  1.142857         1.0   0.428571   3.142857   0.285714   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  
0  

Last 15 Averages

In [120]:
if games_played < 15:
    carry_over_games = 15 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_15_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_15_game_num = games_played - 15
    last_15_df = correct_columns_df.iloc[last_15_game_num:]

print(last_15_df)

last_15_stats = {}

last_15_stats['ab_last_15'] = last_15_df['AB'].mean()
last_15_stats['h_last_15'] = last_15_df['H'].mean()
last_15_stats['r_last_15'] = last_15_df['R'].mean()
last_15_stats['rbi_last_15'] = last_15_df['RBI'].mean()
last_15_stats['hr_last_15'] = last_15_df['HR'].mean()
last_15_stats['tb_last_15'] = last_15_df['TB'].mean()
last_15_stats['bb_last_15'] = last_15_df['BB'].mean()
last_15_stats['k_last_15'] = last_15_df['SO'].mean()
last_15_stats['ba_last_15'] = round(last_15_df['H'].sum() / last_15_df['AB'].sum(), 3)
last_15_stats['obp_last_15'] = round((last_15_df['H'].sum() + last_15_df['BB'].sum() + last_15_df['HBP'].sum()) / last_15_df['AB'].sum(), 3)
last_15_stats['slg_last_15'] = round( last_15_df['TB'].sum() / last_15_df['AB'].sum(), 3)

last_15_avg_df = pd.DataFrame([last_15_stats])

print(last_15_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
42        43  2024-06-05   3  0  0    2   0   0   0   0   0   0  0.247  0.320   
43        44  2024-06-06   4  2  1    2   0   0   1   5   0   1  0.253  0.322   
44        45  2024-06-07   3  2  0    0   0   0   0   2   1   1  0.261  0.332   
45        46  2024-06-08   4  0  0    0   0   0   0   0   0   0  0.255  0.324   
46        47  2024-06-09   5  3  2    2   0   0   0   3   1   0  0.265  0.335   
47        48  2024-06-10   4  2  0    1   1   0   0   3   0   1  0.270  0.338   
48        49  2024-06-11   3  1  0    0   0   0   0   1   1   1  0.271  0.342   
49        50  2024-06-12   6  5  3    1   0   0   0   5   0   0  0.290  0.356   
50        51  2024-06-13   4  3  2    3   0   0   1   6   0   0  0.299  0.363   
51        52  2024-06-14   4  1  2    0   0   0   0   1   1   0  0.298  0.364   
52        53  2024-06-16   5  3  2    3   0   0   2   9   0   1  0.306  0.369   
53        54  2024-06-16   5

Last 30 Averages

In [121]:
if games_played < 30:
    carry_over_games = 30 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_30_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_30_game_num = games_played - 30
    last_30_df = correct_columns_df.iloc[last_30_game_num:]

print(last_30_df)

last_30_stats = {}

last_30_stats['ab_last_30'] = last_30_df['AB'].mean()
last_30_stats['h_last_30'] = last_30_df['H'].mean()
last_30_stats['r_last_30'] = last_30_df['R'].mean()
last_30_stats['rbi_last_30'] = last_30_df['RBI'].mean()
last_30_stats['hr_last_30'] = last_30_df['HR'].mean()
last_30_stats['tb_last_30'] = last_30_df['TB'].mean()
last_30_stats['bb_last_30'] = last_30_df['BB'].mean()
last_30_stats['k_last_30'] = last_30_df['SO'].mean()
last_30_stats['ba_last_30'] = round(last_30_df['H'].sum() / last_30_df['AB'].sum(), 3)
last_30_stats['obp_last_30'] = round((last_30_df['H'].sum() + last_30_df['BB'].sum() + last_30_df['HBP'].sum()) / last_30_df['AB'].sum(), 3)
last_30_stats['slg_last_30'] = round( last_30_df['TB'].sum() / last_30_df['AB'].sum(), 3)

last_30_avg_df = pd.DataFrame([last_30_stats])

print(last_30_avg_df)

    game_num   date_game  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA    OBP  \
27        28  2024-05-18   2  1  0    0   0   0   0   1   1   0  0.268  0.368   
28        29  2024-05-19   4  1  0    0   0   0   0   1   0   3  0.267  0.364   
29        30  2024-05-20   4  1  1    2   0   0   1   4   0   1  0.267  0.361   
30        31  2024-05-21   3  1  1    0   0   1   0   3   1   0  0.269  0.365   
31        32  2024-05-22   4  1  1    1   0   0   1   4   0   0  0.268  0.362   
32        33  2024-05-24   4  2  0    0   1   0   0   3   0   1  0.276  0.366   
33        34  2024-05-25   2  0  1    1   0   0   0   0   1   1  0.271  0.362   
34        35  2024-05-26   4  2  1    1   1   0   0   3   0   1  0.279  0.366   
35        36  2024-05-27   4  0  0    0   0   0   0   0   0   3  0.270  0.356   
36        37  2024-05-28   4  1  1    0   1   0   0   2   0   1  0.269  0.353   
37        38  2024-05-29   5  0  0    0   0   0   0   0   0   1  0.259  0.342   
38        39  2024-05-30   4

Merge Streak Data

In [122]:
streak_data_df = pd.concat([last_7_avg_df, last_15_avg_df, last_30_avg_df], axis=1)
print(streak_data_df)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0        4.0  1.714286  1.142857         1.0   0.428571   3.142857   0.285714   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  ab_last_15  h_last_15  \
0  0.285714      0.429         0.5       0.786         4.0        1.8   

   r_last_15  rbi_last_15  hr_last_15  tb_last_15  bb_last_15  k_last_15  \
0   0.933333          1.0    0.266667    2.733333    0.333333        0.4   

   ba_last_15  obp_last_15  slg_last_15  ab_last_30  h_last_30  r_last_30  \
0        0.45        0.533        0.683    3.866667   1.333333        0.7   

   rbi_last_30  hr_last_30  tb_last_30  bb_last_30  k_last_30  ba_last_30  \
0     0.833333         0.2    2.266667    0.266667   0.766667       0.345   

   obp_last_30  slg_last_30  
0        0.414        0.586  


Merge Season Averages and Streak Data



In [123]:
batter_season_stats_df = pd.concat([streak_data_df, season_stats_df], axis=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(batter_season_stats_df)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  bb_last_7  \
0        4.0  1.714286  1.142857         1.0   0.428571   3.142857   0.285714   

   k_last_7  ba_last_7  obp_last_7  slg_last_7  ab_last_15  h_last_15  \
0  0.285714      0.429         0.5       0.786         4.0        1.8   

   r_last_15  rbi_last_15  hr_last_15  tb_last_15  bb_last_15  k_last_15  \
0   0.933333          1.0    0.266667    2.733333    0.333333        0.4   

   ba_last_15  obp_last_15  slg_last_15  ab_last_30  h_last_30  r_last_30  \
0        0.45        0.533        0.683    3.866667   1.333333        0.7   

   rbi_last_30  hr_last_30  tb_last_30  bb_last_30  k_last_30  ba_last_30  \
0     0.833333         0.2    2.266667    0.266667   0.766667       0.345   

   obp_last_30  slg_last_30  ab_per_game  h_per_game  r_per_game  \
0        0.414        0.586     3.701754    1.140351    0.631579   

   rbi_per_game  hr_per_game  tb_per_game  bb_per_game  k_per_game  ba_szn  \
0      0.614

Fetch Starting Pitcher Season Data


MASSIVE:

PLAN FOR FUTURE FOR PITCHER, BVP, BVOPP, OPP, BVB Data

DESIGN THE 'Starting Pitcher Efficiency Index'

Step 1: SP Data
    - Gather Data such as:
        - ip per game
        - batters faced per game
        - plate appearances per game
        - pitches per game
        - at bats per game

Step 2: Include these new stat categories:
    - Batter Vs Opponent - Previous Batter stats from each game against the opponent this season
    - Team Vs Opponent Stats - Avg Team Stats for Each Final Boxscore of Previous Season Matchups
    - Pitcher vs Team Stats - Avg Pitcher Stats Vs Batter's team from previous mathcups
    - Batter Vs Pitcher - Batter Vs Pitcher Stats from IP or PA from each game featuring the two
    - Batter Vs Bullpen (Batter Vs Team - Batter Vs Pitcher)

    Goal is to determine how valuable the batter's plate appearences with the starting pitcher compared to the batter's plate appearances with the relievers and closers.
    What percentage of hits come from the starting pitcher vs the relievers. Want to try to predict the number of plate appearnaces the batter will get during a game with this starting pitcher, and then use this value to predict hits by comparing the likelihood the batter will get a hit within the plate appearances against the starting pitcher and the likelihood the batter will get a hit within the plate appearances against the relievers and closers. Then using these averages, predict how likely the batter is to get a hit in this game. Basically instead of just using starting pitcher, want to treat both the starting pitcher and other pitchers as an entire game to predict hits. This will be done by leveraging the innings pitched by the starter and other pitchers with the stats of starter v batter and bullpen v batter


Format Pitcher URL

In [124]:
sp_name_parts = sPitcher_name.split()

sp_first_name = sp_name_parts[0].lower()
sp_last_name = sp_name_parts[1].lower()

initial = sp_last_name[:1]

full_name = f'{sp_first_name} {sp_last_name}'

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                formatted_pitcher_key = player_link[11:20]

print(formatted_pitcher_key)

estesjo01


In [139]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_pitcher_key}&t=p&year={year}#pitching_gamelogs'

response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('pitching_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    sp_df = pd.DataFrame(data)

    sp_df.rename(columns={
            'earned_run_avg': 'ERA',
            'batters_faced': 'BF',
            'pitches': 'P',
            'strikes_total': 'S',
        }, inplace=True)

    sp_all_columns = sp_df.columns.tolist()

    sp_columns_to_keep = ['game_num', 'date_game', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'BF', 'P', 'S', 'AB', '2B', '3B']

    sp_columns_to_drop = [column for column in sp_all_columns if column not in sp_columns_to_keep]

    sp_column_df = sp_df.drop(sp_columns_to_drop, axis=1)

    sp_columns_to_convert_int = ['game_num', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'BF', 'P', 'S', 'AB', '2B', '3B']

    sp_columns_to_convert_float = ['IP', 'ERA']

    for column in sp_columns_to_convert_int:
        sp_column_df[column] = sp_column_df[column].astype(int)

    for column in sp_columns_to_convert_float:
        sp_column_df[column] = sp_column_df[column].astype(float)

    sp_column_df['WHIP'] = sp_column_df.apply(lambda row: (row['H'] + row['BB']) / row['IP'], axis=1)

    sp_column_df['TB'] = sp_column_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

    sp_column_df['IP'] = sp_column_df['IP'].apply(lambda ip: int(ip) + (1/3) if round(ip % 1, 1) == 0.1 else int(ip) + (2/3) if round(ip % 1, 1) == 0.2 else ip)

    sp_column_df['TO'] = sp_column_df.apply(lambda row: row['IP']*3, axis=1)

    sp_column_df['S%'] = sp_column_df.apply(lambda row: row['S'] / row['P'], axis=1)

    sp_column_df['BAA'] = sp_column_df.apply(lambda row: row['H'] / row['AB'], axis=1)

    sp_column_order = ['game_num', 'date_game', 'IP', 'P', 'S', 'S%', 'BF', 'AB', 'H', '2B', '3B', 'R', 'ER', 'HR', 'TB', 'BB', 'SO', 'TO', 'HBP', 'ERA', 'WHIP', 'BAA']
                       
    sp_complete_df = sp_column_df[sp_column_order]

    print(sp_complete_df)

   game_num   date_game        IP   P   S        S%  BF  AB  H  2B  3B  R  ER  \
0         1  2024-05-11  5.000000  61  40  0.655738  17  17  2   1   0  1   1   
1         2  2024-05-16  3.666667  89  56  0.629213  23  18  8   3   0  8   8   
2         3  2024-05-23  7.000000  93  73  0.784946  27  27  7   4   0  4   4   
3         4  2024-05-29  5.000000  87  60  0.689655  20  17  2   1   1  1   1   
4         5  2024-06-05  6.333333  78  58  0.743590  20  20  1   1   0  0   0   
5         6  2024-06-10  5.000000  96  63  0.656250  24  22  8   3   0  3   3   
6         7  2024-06-16  2.666667  65  44  0.676923  18  16  8   4   0  6   6   

   HR  TB  BB  SO    TO  HBP   ERA      WHIP       BAA  
0   1   6   0   5  15.0    0  1.80  0.400000  0.117647  
1   1  14   4   4  11.0    0  9.35  3.750000  0.444444  
2   0  11   0   6  21.0    0  7.47  1.000000  0.259259  
3   0   5   2   5  15.0    1  6.10  0.800000  0.117647  
4   0   2   0   5  19.0    0  4.67  0.163934  0.050000  
5   2  17

Pitcher Season Stats Up to Desired Date

In [143]:
sp_season_stats = {}

sp_game_data = sp_complete_df.iloc[-1]

sp_games_started = sp_game_data['game_num']

print(sp_games_started)

sp_season_stats['sp_games_started'] = sp_games_started
sp_season_stats['sp_ip_per_game'] = sp_complete_df['IP'].mean()
sp_season_stats['sp_p_per_game'] = sp_complete_df['P'].mean()
sp_season_stats['sp_s_per_game'] = sp_complete_df['S'].mean()
sp_season_stats['sp_s%'] = sp_complete_df['S'].sum() / sp_complete_df['P'].sum()
sp_season_stats['sp_bf_per_game'] = sp_complete_df['BF'].mean()
sp_season_stats['sp_ab_per_game'] = sp_complete_df['AB'].mean()
sp_season_stats['sp_h_per_game'] = sp_complete_df['H'].mean()
sp_season_stats['sp_r_per_game'] = sp_complete_df['R'].mean()
sp_season_stats['sp_er_per_game'] = sp_complete_df['ER'].mean()
sp_season_stats['sp_hr_per_game'] = sp_complete_df['HR'].mean()
sp_season_stats['sp_tb_per_game'] = sp_complete_df['TB'].mean()
sp_season_stats['sp_bb_per_game'] = sp_complete_df['BB'].mean()
sp_season_stats['sp_k_per_game'] = sp_complete_df['SO'].mean()
sp_season_stats['sp_k%'] = sp_complete_df['SO'].sum() / sp_complete_df['BF'].sum()
sp_season_stats['sp_to_per_game'] = sp_complete_df['TO'].mean()
sp_season_stats['sp_era'] = sp_game_data['ERA']
sp_season_stats['sp_whip_per_game'] = sp_complete_df['WHIP'].mean()
sp_season_stats['sp_whip_szn'] = ((sp_complete_df['H'].sum() + sp_complete_df['BB'].sum() ) / sp_complete_df['IP'].sum())
sp_season_stats['sp_baa'] = sp_complete_df['H'].sum() / sp_complete_df['AB'].sum()

sp_season_stats_df = pd.DataFrame([sp_season_stats])

print(sp_season_stats_df)

7
   sp_games_started  sp_ip_per_game  sp_p_per_game  sp_s_per_game     sp_s%  \
0                 7        4.952381      81.285714      56.285714  0.692443   

   sp_bf_per_game  sp_ab_per_game  sp_h_per_game  sp_r_per_game  \
0       21.285714       19.571429       5.142857       3.285714   

   sp_er_per_game  sp_hr_per_game  sp_tb_per_game  sp_bb_per_game  \
0        3.285714        0.857143       10.428571        1.285714   

   sp_k_per_game     sp_k%  sp_to_per_game  sp_era  sp_whip_per_game  \
0       4.285714  0.201342       14.857143    5.97          1.743549   

   sp_whip_szn    sp_baa  
0     1.298077  0.262774  


Format Batter vs Starting Pitcher URL :P

In [127]:
formatted_batter_name = player_name.replace(' ', '%20')

team_schedule = api.schedule(start_date=season_start_date, end_date=date, team=team_id, opponent=opp_id)

if len(team_schedule) > 0:

    games_vs_opp = []
    games_vs_sp = []

    for game in team_schedule:
        if game['game_type'] != 'S' and game['status'] != 'Postponed':
            for log in api.get('person_stats', {'personId': player_id, 'gamePk': game['game_id']})['stats'][0]['splits']:
                if log['group'] == 'hitting' and log['stat']['plateAppearances'] > 0:
                    games_vs_opp.append(game['game_id'])
                    if game['home_id'] == opp_id:
                        if game['home_probable_pitcher'] == sPitcher_name:
                            games_vs_sp.append(game['game_date'])
                    elif game['away_id'] == opp_id:
                        if game['away_probable_pitcher'] == sPitcher_name:
                            games_vs_sp.append(game['game_date'])
    
    num_games_vs_sp = len(games_vs_sp)
    num_games_vs_opp = len(games_vs_opp)

    game_strings = [f'%22{game_vs_opp}%22' for game_vs_opp in games_vs_opp]

    comma_game_strings =f'[' + ','.join(game_strings) + ']'

    if num_games_vs_sp == 0:
        bvp_url = None
    else:
        bvp_url = f'https://baseballsavant.mlb.com/illustrator?playerId={player_id}&playerType=batter&name={formatted_batter_name}&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22{sPitcher_id}%22]&balls=[]&battedBallType=[]&years=[{year}]&strikes=[]&outs=[]&selectedGames=' + comma_game_strings + '&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7'


else:
    num_games_vs_sp = 0
    num_games_vs_opp = 0
    games_vs_opp = None
    games_vs_sp = None
    bvp_url = None

print(num_games_vs_sp)
print(num_games_vs_opp)

print(bvp_url)
print(games_vs_opp)
print(games_vs_sp)


1
4
https://baseballsavant.mlb.com/illustrator?playerId=621043&playerType=batter&name=carlos%20correa&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22683155%22]&balls=[]&battedBallType=[]&years=[2024]&strikes=[]&outs=[]&selectedGames=[%22745894%22,%22745892%22,%22745895%22,%22745891%22]&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7
[745894, 745892, 745895, 745891]
['2024-06-16']


Gather Raw BVP Data

In [128]:
if bvp_url is not None:
    bvp_raw_data = {}

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    driver.get(bvp_url)

    driver.implicitly_wait(10)

    stats_boxes = driver.find_elements(By.CLASS_NAME, 'stats-box')

    for stats_box in stats_boxes:
        stat_type = stats_box.find_element(By.CLASS_NAME, 'stats-title')
        stat_value = stats_box.find_element(By.CLASS_NAME, 'stats-value')
        bvp_raw_data[stat_type.text] = stat_value.text


    driver.quit()

    for key in bvp_raw_data.keys():
        bvp_raw_data[key] = float(bvp_raw_data[key])

else:
    bvp_raw_data = None

print(bvp_raw_data)

{'Pitches': 3.0, 'PA': 2.0, 'H': 1.0, '1B': 1.0, '2B': 0.0, '3B': 0.0, 'HR': 0.0, 'AVG': 0.5, 'SO': 0.0, 'wOBA': 0.444, 'K%': 0.0, 'EV': 101.3, 'LA': -2.0}


Manipulate bvp data

TO DO: Remove normalization to 9 innings and keep as total per game.
Consider: switching to per plate appearance
Consider: instead of just looking at bvp, look at entire game where that pitcher was sp
            - record stats per plate appearance against starting pitcher
            - record stats per plate appearance against entire bullpen
            - most importantly, determine avg of plate appearances against sp per game and stats associated with those plate appearances
            - big idea: want to determine how many plate appearances the batter will get vs this sp, and then determine the likelihood of player getting a hit within those plate appearances.
            - Then want to determine how many plate appearances the batter will get after sp, then then determine the likelihood of a player getting a hit within those plate appearances.
            - Combine both of these stats to predict the amount of plate appearances the batter will get per game, and how likely they are to get a hit given the split of those plate appearances 

In [132]:
bvp_data = {'bvp_games_faced': 0,
            'bvp_sp_ip_per_game': 0.0,
            'bvp_sp_p_per_game': 0.0,
            'bvp_b_pa_per_game': 0.0,
            'bvp_b_ab_per_game': 0.0,
            'bvp_b_h_per_game': 0.0,
            'bvp_b_hr_per_game': 0.0,
            'bvp_b_tb_per_game': 0.0,
            'bvp_b_k_per_game': 0.0,
            'bvp_sp_k%': 0.0,
            'bvp_sp_woba': 0.0,
            'bvp_b_ba': 0.0,
            'bvp_b_slg': 0.0
            }

if bvp_raw_data is not None:
    bvp_total_data = {}
    sp_ip = 0.0

    for date in games_vs_sp:
        sp_ip += float(sp_column_df[sp_column_df['date_game'] == date]['IP'].iloc[0])

    sp_ip_per_bvp = round(sp_ip / num_games_vs_sp, 3)
    bvp_total_data['bvp_sp_ip_per_game'] = sp_ip_per_bvp

    bvp_total_data['bvp_sp_p_per_game'] = round((bvp_raw_data['Pitches'] * 9.0) / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_b_pa_per_game'] = round((bvp_raw_data['PA'] * 9.0) / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_b_h_per_game'] = round((bvp_raw_data['H'] * 9.0) / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_1B_adj'] = round((bvp_raw_data['1B'] * 9.0) / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_2B_adj'] = round((bvp_raw_data['2B'] * 9.0) / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_3B_adj'] = round((bvp_raw_data['3B'] * 9.0) / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_b_hr_per_game'] = round((bvp_raw_data['HR'] * 9.0) / sp_ip_per_bvp, 3)
    bvp_total_data['bvp_b_k_per_game'] = round((bvp_raw_data['SO'] * 9.0) / sp_ip_per_bvp, 3)
    
    bvp_total_data['bvp_b_tb_per_game'] = round((bvp_total_data['bvp_1B_adj'] + 2 * bvp_total_data['bvp_2B_adj']
                                                  + 3 * bvp_total_data['bvp_3B_adj'] + 4 * bvp_total_data['bvp_b_hr_per_game']), 3)
    bvp_total_data['bvp_b_ab_per_game'] = round(((bvp_raw_data['H'] / bvp_raw_data['AVG']) * 9.0) / sp_ip_per_bvp, 3)

    bvp_data['bvp_b_ba'] = bvp_raw_data['AVG']
    bvp_data['bvp_b_slg'] = round((bvp_total_data['bvp_b_tb_per_game'] / bvp_total_data['bvp_b_ab_per_game']), 3)
    bvp_data['bvp_sp_k%'] = bvp_raw_data['K%']
    bvp_data['bvp_sp_woba'] = bvp_raw_data['wOBA']

    for i in ['bvp_1B_adj', 'bvp_2B_adj', 'bvp_3B_adj']:
        bvp_total_data.pop(i)
    
    for total in bvp_total_data.keys():
        bvp_data[total] = round(bvp_total_data[total] / num_games_vs_sp, 3)
    
    bvp_data['bvp_games_faced'] = num_games_vs_sp
    
bvp_data_df = pd.DataFrame([bvp_data]).astype(float)

print(bvp_data_df)
        

   bvp_games_faced  bvp_sp_ip_per_game  bvp_sp_p_per_game  bvp_b_pa_per_game  \
0              1.0               2.667             10.124              6.749   

   bvp_b_ab_per_game  bvp_b_h_per_game  bvp_b_hr_per_game  bvp_b_tb_per_game  \
0              6.749             3.375                0.0              3.375   

   bvp_b_k_per_game  bvp_sp_k%  bvp_sp_woba  bvp_b_ba  bvp_b_slg  
0               0.0        0.0        0.444       0.5        0.5  


Gather b_vs_opp Data

In [136]:
bv_opp_data_dict = {}

if num_games_vs_opp > 0:

    bv_opp_raw_data = []

    for game in games_vs_opp:
        game_vs_opp_log = api.get('person_stats', {'personId': player_id, 'gamePk': game})['stats'][0]['splits']
        game_vs_opp_data = {}
        for group in game_vs_opp_log:
            if group['group'] == 'hitting':
                for s in group['stat']:
                    game_vs_opp_data[s] = group['stat'][s]
        bv_opp_raw_data.append(game_vs_opp_data)
    
    bv_opp_raw_data_df = pd.DataFrame(bv_opp_raw_data)

    bv_opp_raw_data_df.rename(columns={
        'runs': 'bv_opp_r',
        'doubles': '2B',
        'triples': '3B',
        'homeRuns': 'bv_opp_hr',
        'strikeOuts': 'bv_opp_k',
        'baseOnBalls': 'bv_opp_bb',
        'hits': 'bv_opp_h',
        'hitByPitch': 'HBP',
        'atBats': 'bv_opp_ab',
        'totalBases': 'bv_opp_tb',
        'rbi': 'bv_opp_rbi',
        'sacFlies': 'SF'
    }, inplace=True)

    bv_opp_columns = bv_opp_raw_data_df.columns.to_list()

    bv_opp_columns_to_keep = ['bv_opp_ab', 'bv_opp_h', 'bv_opp_r', 'bv_opp_rbi', '2B', '3B', 'bv_opp_hr', 'bv_opp_tb', 'bv_opp_bb', 'bv_opp_k', 'HBP', 'SF']

    bv_opp_columns_to_drop = [column for column in bv_opp_columns if column not in bv_opp_columns_to_keep]

    bv_opp_columns_df = bv_opp_raw_data_df.drop(bv_opp_columns_to_drop, axis=1)

    bv_opp_columns_df.astype(int)

    for column in bv_opp_columns_df.columns.to_list():
        bv_opp_data_dict[column] = bv_opp_columns_df[column].mean()

    bv_opp_data_dict['bv_opp_ba'] = round(bv_opp_data_dict['bv_opp_h'] / bv_opp_data_dict['bv_opp_ab'], 3)
    bv_opp_data_dict['bv_opp_obp'] = round((bv_opp_data_dict['bv_opp_h'] + bv_opp_data_dict['bv_opp_bb'] + bv_opp_data_dict['HBP']) / (
        bv_opp_data_dict['bv_opp_ab'] + bv_opp_data_dict['bv_opp_bb'] + bv_opp_data_dict['HBP'] + bv_opp_data_dict['SF']), 3)
    bv_opp_data_dict['bv_opp_slg'] = round(bv_opp_data_dict['bv_opp_tb'] / bv_opp_data_dict['bv_opp_ab'], 3)

    for key in ['2B', '3B', 'HBP', 'SF']:
        bv_opp_data_dict.pop(key)
    
    bv_opp_data_dict['bv_opp_games_played'] = num_games_vs_opp

else:
    bv_opp_data_dict = {'bv_opp_games_played': 0, 'bv_opp_ab': 0, 'bv_opp_h': 0,'bv_opp_r': 0, 'bv_opp_rbi': 0, 'bv_opp_hr': 0, 'bv_opp_tb': 0, 'bv_opp_bb': 0, 'bv_opp_k': 0,
                         'bv_opp_ba': 0.0, 'bv_opp_obp': 0.0, 'bv_opp_slg': 0.0}

bv_opp_data_df = pd.DataFrame([bv_opp_data_dict])

bv_opp_column_order = ['bv_opp_games_played', 'bv_opp_ab', 'bv_opp_h', 'bv_opp_r', 'bv_opp_rbi', 'bv_opp_hr', 'bv_opp_tb', 'bv_opp_bb', 'bv_opp_k', 'bv_opp_ba', 'bv_opp_obp', 'bv_opp_slg']

bv_opp_df = bv_opp_data_df[bv_opp_column_order]

print(bv_opp_df)

   bv_opp_games_played  bv_opp_ab  bv_opp_h  bv_opp_r  bv_opp_rbi  bv_opp_hr  \
0                    4        4.5      2.25       1.5        1.75       0.75   

   bv_opp_tb  bv_opp_bb  bv_opp_k  bv_opp_ba  bv_opp_obp  bv_opp_slg  
0        4.5       0.25       0.5        0.5       0.526         1.0  


Fetch Umpire Stats
- If Home Plate Ump announced
    - Return data for only HP ump 
- If Home Plate Ump not yet announced
    - Fetch data for all 4 umps
    - hp_ump stats = avg of 4 umps stats

Fetch Venue Stats *WIP